## Embedding with BGE's model

A `SentenceTransformer` model instantiated from `NVIDIA-Embed-v2` will return tensors or np arrays but can only take a string input or a list containing a single string. It cannot accept a list of strings.

In [1]:
from sentence_transformers import SentenceTransformer
import torch

model_name = "BAAI/bge-small-en"
sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
]
device = 'mps' # mps was producing a dimension error on batch input to model.encode 
print(f"Using device: {device}")
model = SentenceTransformer(
    model_name, trust_remote_code=True, device=device)
print(model)

# Encoding a list of strings
try:
    embeddings = model.encode(
        sentences, convert_to_tensor=True, normalize_embeddings=False)
    print(f"Batch embeddings shape: {embeddings.shape}")
except Exception as e:
    print(f"Error: {e}")

Using device: mps


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)
Batch embeddings shape: torch.Size([3, 384])


In [2]:
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings

client = chromadb.PersistentClient(path='./vector_stores/bge/')
print(client)

# Set up embedding model
class BgeChromaEmbedder(EmbeddingFunction):
    def __init__(self, embedding_fn):
        self._encode = embedding_fn

    def __call__(self, input: Documents) -> Embeddings:
        return self._encode(input)

embedding_lambda = lambda docs: model.encode(docs, convert_to_numpy=True, normalize_embeddings=False)

embedder = BgeChromaEmbedder(embedding_lambda)
result = embedder(['hi there', 'hello world'])
print(f"Return object type: {type(result)}")
print(f"Inner object type: {type(result[0])}")
print(result[0].shape)

Return object type: <class 'list'>
Inner object type: <class 'numpy.ndarray'>
(384,)


In [3]:
import os
import json

METRICS = ['l2', 'cosine', 'ip']
PATH_TO_DATA = 'data/processed_for_chroma/research'
FILENAMES = os.listdir(PATH_TO_DATA)
data = dict()

for filename in FILENAMES:
    with open(f'{PATH_TO_DATA}/{filename}', 'r') as file:
        data[os.path.splitext(os.path.basename(filename))[0]] = json.load(file)

print(data.keys())
data['Astro_Research'].keys()

dict_keys(['Earth_Science_Research', 'Planetary_Research', 'Astro_Research'])


dict_keys(['documents', 'metadatas', 'ids'])

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
doc = data['Astro_Research']['documents'][0]
print(f"Length of doc string: {len(doc)}")


Length of doc string: 218019


In [5]:
abstract = data['Astro_Research']['metadatas'][0]['abstract']
print(abstract)
embedder([abstract])

We present a new model for computing the spectral evolution of stellar populations at ages between 1 × 10<SUP>5</SUP> and 2 × 10<SUP>10</SUP> yr at a resolution of 3 Å across the whole wavelength range from 3200 to 9500 Å for a wide range of metallicities. These predictions are based on a newly available library of observed stellar spectra. We also compute the spectral evolution across a larger wavelength range, from 91 Å to 160 μm, at lower resolution. The model incorporates recent progress in stellar evolution theory and an observationally motivated prescription for thermally pulsing stars on the asymptotic giant branch. The latter is supported by observations of surface brightness fluctuations in nearby stellar populations. We show that this model reproduces well the observed optical and near-infrared colour-magnitude diagrams of Galactic star clusters of various ages and metallicities. Stochastic fluctuations in the numbers of stars in different evolutionary phases can account for 

[array([-6.78137466e-02, -2.68423390e-02, -1.86776333e-02, -1.07058212e-02,
         5.27977645e-02,  2.23929957e-02, -3.07627786e-02,  2.08602417e-02,
         7.53328064e-03, -1.90267470e-02,  5.52427582e-02, -7.90005550e-02,
         3.87193146e-03,  4.22988646e-03, -1.80166457e-02,  1.99470446e-02,
        -2.54089721e-02, -1.28044495e-02, -1.41485874e-02,  1.54637527e-02,
         2.16971338e-02, -1.14673860e-02, -1.38329323e-02, -1.53478906e-02,
         2.63023153e-02,  5.25402650e-02, -3.32830474e-02, -5.92271499e-02,
        -6.10397570e-02, -2.73399532e-01, -2.55301967e-03, -8.91098287e-03,
         3.99910770e-02, -2.41717678e-02, -2.15523876e-02, -3.36218253e-02,
         2.93930876e-03,  1.27296252e-02, -1.88713856e-02,  1.88691951e-02,
        -2.43197735e-02, -7.46643450e-03,  1.35477735e-02, -8.24323017e-03,
        -4.27904800e-02, -5.05801756e-03, -3.88098136e-02,  1.07834721e-02,
        -1.56607740e-02, -3.81065831e-02, -2.21574791e-02, -3.02540865e-02,
        -6.0

In [15]:
res = embedder(doc[:200000])
print(res[0].shape)

(384,)


In [16]:
# Reset collections
collections = client.list_collections()

# Delete each collection
for collection in collections:
    client.delete_collection(name=collection.name)

print("All collections have been deleted.")

All collections have been deleted.


In [17]:
collection = client.create_collection(
    name='testing-insert',
    embedding_function=embedder,
    metadata={"hnsw:space": 'l2'}
)



In [18]:
collection.add(
    documents=data['Astro_Research']['documents'][0:1],
    metadatas=data['Astro_Research']['metadatas'][0:1],
    ids=data['Astro_Research']['ids'][0:1]
)

In [ ]:
ADD_INCREMENT = 10

for metric in METRICS:
    clean_model_name = model_name.replace('/', '-')
    collection_name = f"{clean_model_name}_{metric}_no-norm"
    print(f"Creating collection: {collection_name}...")

    collection = client.create_collection(
        name=collection_name,
        embedding_function=embedder,
        metadata={"hnsw:space": metric}
    )
    for journal, records in data.items():
        print(f"  Adding {journal} records...")

        # Add records 10 at a time
        for i in range(len(records['documents']) // ADD_INCREMENT):
            s = slice(i*ADD_INCREMENT, (i+1)*ADD_INCREMENT, 1)
            collection.add(
                documents=records['documents'][s],
                metadatas=records['metadatas'][s],
                ids=records['ids'][s]
            )
    print(f"Finished {collection}")

In [ ]:
# Test the dimensionality of the embeddings
collection = client.get_collection(name=f"{model_name}_cosine_no-norm")
chroma_embedding = collection.query(
    query_texts=["the sun is a star"],
    n_results=1,
    include=["embeddings", "documents"]
)

print(type(chroma_embedding['embeddings']))
print(type(chroma_embedding['embeddings'][0]))